In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import folium
from matplotlib import pyplot as plt
from geopy.geocoders import Nominatim
from folium import Marker
import warnings 
warnings.filterwarnings('ignore')

In [2]:
geolocator = Nominatim(user_agent="Preço Commodities")
location = geolocator.geocode("Kyiv")

In [3]:
point = location.point
print("Latitude:", point.latitude)
print("Longitude:", point.longitude)

Latitude: 50.4500336
Longitude: 30.5241361


In [60]:
file_path = "../Dados/wfp_food_prices_ukr.csv"

attrs = ['market', 'price', 'date', 'commodity', 'priceflag']

regioes = pd.read_csv(file_path,  dtype='unicode')
regioes = regioes[1:][attrs]

rice_prices = regioes[(regioes['commodity'] == 'Rice') & (regioes['priceflag'] == 'actual')]

rice_prices['year'] = rice_prices['date'].str[0:4]

rice_pre2022 = rice_prices[rice_prices['year'] < '2022']
rice_post2022 = rice_prices[rice_prices['year'] >= '2022']

rice_pre2022 = rice_pre2022.groupby('market')[['price']].max().reset_index()
rice_post2022 = rice_post2022.groupby('market')[['price']].max().reset_index()

display(rice_pre2022)
display(rice_post2022)

,market,price
0,Cherkasy,9.94
1,Chernihiv,9.94
2,Chernivtsi,9.28
3,Crimea,7.76
4,Dnipropetrovsk,9.81
5,Donetsk,9.48
6,Iv.-Frankivsk,9.88
7,Kharkiv,9.88
8,Kherson,9.99
9,Khmelnytskiy,9.67


,market,price
0,Cherkasy,57.33
1,Chernihiv,58.01
2,Chernivtsi,63.75
3,Dnipropetrovsk,61.18
4,Donetsk,33.78
5,Iv.-Frankivsk,59.06
6,Kharkiv,59.93
7,Kherson,57.82
8,Khmelnytskiy,58.36
9,Kyiv,65.67


In [63]:
def my_geocoder(row):
    try:
        point = geolocator.geocode(row).point
        return pd.Series({'Latitude': point.latitude, 'Longitude': point.longitude})
    except:
        return None
    
#rice_pre2022 = regioes.copy()
rice_pre2022[['Latitude', 'Longitude']] = rice_pre2022.apply(lambda x: my_geocoder(x['market']), axis = 1)

display(rice_pre2022.head())

print("{}% of addresses were geocoded!".format(
    (1 - sum(np.isnan(rice_pre2022["Latitude"])) / len(rice_pre2022)) * 100))

# Drop universities that were not successfully geocoded
rice_pre2022 = rice_pre2022.loc[~np.isnan(rice_pre2022["Latitude"])]
rice_pre2022 = gpd.GeoDataFrame(
    rice_pre2022, geometry=gpd.points_from_xy(rice_pre2022.Longitude, rice_pre2022.Latitude))
rice_pre2022.crs = {'init': 'epsg:4326'}
display(rice_pre2022)

,market,price,Latitude,Longitude
0,Cherkasy,9.94,49.146017,31.227174
1,Chernihiv,9.94,51.272593,31.741723
2,Chernivtsi,9.28,48.286470,25.937653
3,Crimea,7.76,45.283504,34.200819
4,Dnipropetrovsk,9.81,48.662589,34.950172


89.28571428571429% of addresses were geocoded!


,market,price,Latitude,Longitude,geometry
0,Cherkasy,9.94,49.146017,31.227174,POINT (31.22717 49.14602)
1,Chernihiv,9.94,51.272593,31.741723,POINT (31.74172 51.27259)
2,Chernivtsi,9.28,48.286470,25.937653,POINT (25.93765 48.28647)
3,Crimea,7.76,45.283504,34.200819,POINT (34.20082 45.28350)
4,Dnipropetrovsk,9.81,48.662589,34.950172,POINT (34.95017 48.66259)
5,Donetsk,9.48,48.015875,37.801341,POINT (37.80134 48.01588)
6,Iv.-Frankivsk,9.88,48.634829,24.569047,POINT (24.56905 48.63483)
7,Kharkiv,9.88,49.992318,36.231015,POINT (36.23101 49.99232)
8,Kherson,9.99,46.542172,33.407933,POINT (33.40793 46.54217)
10,Kirovohrad,25.12,48.372523,31.783482,POINT (31.78348 48.37252)


In [65]:
# Create a map
m = folium.Map(location=[54, 15], tiles='cartodb positron', zoom_start=2)

# Add points to the map
for idx, row in rice_pre2022.iterrows():
    Marker([row['Latitude'], row['Longitude']], popup=row['market']).add_to(m)
political_countries_url = (
    "http://geojson.xyz/naturalearth-3.3.0/ne_50m_admin_0_countries.geojson"
)
folium.Choropleth(
    geo_data=political_countries_url,
    data=rice_pre2022,
    columns=["market", "price"],
    key_on="feature.properties.name",
).add_to(m)

# Display the map
m

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
file_path = "../Dados/wfp_food_prices_ukr.csv"

regioes = pd.read_csv(file_path,  dtype='unicode')
regioes = regioes[1:]

prices = regioes.groupby('market')[['price']]
dates = regioes.groupby('market')[['date']]
commodity = regioes.groupby('market')[['commodity']]

regioes_final = prices.join(dates).join(commodity)

display(regioes_final.head())
